# Configuration

In [ ]:
CONNECTION_STRING = "mongodb://localhost:27017"
DATABASE_NAME = "off"
COLLECTION_NAME = "products"

# MongDB connection

In [ ]:
from pymongo import MongoClient
from bson.code import Code
import plotly, pymongo
plotly.offline.init_notebook_mode()
from plotly.graph_objs import Bar

client = MongoClient(CONNECTION_STRING)
db = client[DATABASE_NAME]
openfood = db[COLLECTION_NAME]

# Nutrition grade

In [ ]:
mapper = Code("""
    function () {
        if (typeof this.nutrition_grades !== 'undefined' && this.nutrition_grades !== ""){
            emit(this.nutrition_grades, 1);
        }
    }""")
reducer = Code("""
    function (key, values) {
        var total = 0;
        for (var i = 0; i < values.length; i++) {
            total += values[i];
        }
        return total;
    }""")

grades = openfood.inline_map_reduce(mapper, reducer)
print grades

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
objects = [item['_id'] for item in grades] # [a,b,c,d,e]
y_pos = np.arange(len(objects))
count = [item['value'] for item in grades]
 
plt.bar(y_pos, count, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Count')
plt.title('Nutrition Grades')

plt.show()

# Nutrients (100g)

In [ ]:
mapper = Code("""
    function () {
        if (typeof this.nutriments !== 'undefined' && this.nutriments !== "") {
            for (var key in this.nutriments) {
                if (key.match(/.*100g/))
                    emit(key, null);
            }
        }
    }""")
reducer = Code("""
    function (key, values) {
        return key
    }""")

nutriments_100g_fields = openfood.inline_map_reduce(mapper, reducer)
for n in nutriments_100g_fields:
    print n

In [ ]:
for n in nutriments_100g_fields:
    print n['_id']